In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [16]:
# use large scale here to get the output really close to the value in dataset, which is important to Mode.
# large scale doesn't blowup circuit size in Mode, so fine.
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [17]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.mode(x)
# IMPORTANT: Unlike other functions, error of Mode is num_error, NOT the result error.
error = 0
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [18]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [19]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.386570930480957 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.6015625
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10694507807471723898, 15190737326438772030, 17750154044162608345, 421916922553260280]]], 'proof': '019e8a20063406f049746fa9014ee3ccda33f028ffa7cf3c79d8ba979bf84bca2cf1c58080cb7edc3beab05496555b87a926330342fbe0da600652b68f67c9b722e9688a2b02a86cd5849fadb34446dc0e6567a6b504876dc19df81f05adeda81b9f5bfbbb2ccd436d9ba70bc6ed94ac4aff4282e3fc64b31e1e2726a16b4ce81441981cf3497729e9fb7db664ec776d0d5478bdb0f8775ce2d39be1e51e973b2e2d59d4b10a26bf8ff40083b4b65d76cc49268fcb39ddab93355bb2e9938d57301cdf88c4b848ad238c6e4bc74fbdf4117a9d1d60f30708bfe346d7f23050b02b3cd8d3c9af19cec11e4597ad8c5d192b9972806b421db98f0e040b6ebca5b70c7bc02a37168e4dfb917f01057a7294e5825deeec6e1d5fe1f852c7fd8dac150a6605dfdf4d5b69ff1356c54a3a45bf243354f6d57dfe0c2d088dd9343cdb7b242cd798f2ecb5efa9329ebec3ab5d238a8d62c46b6337d22e31b50884f07447

In [20]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 77.6015625
